<a href="https://colab.research.google.com/github/lazy-wav/Data-Analysis/blob/main/PoliDataSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
uploaded = files.upload()

Saving c2_new_model_weights.pt to c2_new_model_weights.pt


In [1]:
import torch
import torch.nn as nn
!pip install transformers==4.28.1
from transformers import AutoModel, BertTokenizerFast

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load BERT and tokenizer
bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Define the architecture again
class BERT_Arch(nn.Module):
    def __init__(self, bert):
        super(BERT_Arch, self).__init__()
        self.bert = bert
        self.dropout = nn.Dropout(0.1)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(768, 512)
        self.fc2 = nn.Linear(512, 2)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, sent_id, mask):
        cls_hs = self.bert(sent_id, attention_mask=mask)['pooler_output']
        x = self.fc1(cls_hs)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

# Initialize and load weights
model = BERT_Arch(bert)
model.load_state_dict(torch.load('c2_new_model_weights.pt', map_location=device))
model.to(device)
model.eval()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (in

BERT_Arch(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=

In [2]:
import pandas as pd
from google.colab import files

uploaded = files.upload()  # Upload

Saving politifact_real.csv to politifact_real.csv


Saving politifact_fake.csv to politifact_fake.csv


In [3]:
import pandas as pd

# Load the CSV files
fake_df = pd.read_csv('politifact_fake.csv')
real_df = pd.read_csv('politifact_real.csv')

# Add labels: 0 for fake, 1 for real
fake_df['label'] = 0
real_df['label'] = 1

# Keep only the title and label columns
fake_df = fake_df[['title', 'label']]
real_df = real_df[['title', 'label']]

# Combine and shuffle the datasets
combined_df = pd.concat([fake_df, real_df], ignore_index=True)
combined_df = combined_df.sample(frac=1).reset_index(drop=True)

# Extract text and labels
text_data = combined_df['title'].tolist()
labels = combined_df['label'].tolist()

# Quick peek at the combined data
print(combined_df.head())


                                               title  label
0  WE WILL RIOT! Michelle Obama’s Mom Will Receiv...      0
1  The pro-Russia, pro-weed, pro-Assange GOP cong...      0
2                                                NaN      1
3                                                NaN      1
4                                                NaN      1


In [6]:
from sklearn.metrics import classification_report

# Ensure all text entries are strings
text_data = [str(t) if t is not None else '' for t in text_data]

# Tokenize
tokens = tokenizer.batch_encode_plus(
    text_data,
    max_length=64,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

input_ids = tokens['input_ids'].to(device)
attention_mask = tokens['attention_mask'].to(device)

# Get predictions
with torch.no_grad():
    outputs = model(input_ids, attention_mask)
    predictions = torch.argmax(outputs, dim=1).cpu().numpy()

# Convert true labels to numpy array
true_labels = torch.tensor(labels).numpy()

# Print classification report
print(classification_report(true_labels, predictions, target_names=['False', 'True']))


              precision    recall  f1-score   support

       False       0.32      0.46      0.38       473
        True       0.57      0.43      0.49       797

    accuracy                           0.44      1270
   macro avg       0.45      0.44      0.43      1270
weighted avg       0.48      0.44      0.45      1270

